In [1]:
import os
import sys
sys.path.append('..')

import cv2
import numpy as np
import torch
from kornia import image_to_tensor, tensor_to_image

from utils.io import load_yaml
from lightning_modules import DeepImageHomographyEstimationModuleBackbone

endoscopic_view_model_prefix = '/home/martin/Tresors/homography_imitation_learning_logs/endoscopy_view/resnet/34/version_0'
# endoscopic_view_model = 'checkpoints/epoch=229-step=108559.ckpt'
endoscopic_view_model = 'checkpoints/epoch=197-step=93455.ckpt'

non_endoscopic_view_model_prefix = '/home/martin/Tresors/homography_imitation_learning_logs/ae_cai/resnet/48/25/34/version_0'
non_endoscopic_view_model = 'checkpoints/epoch=99-step=47199.ckpt'

endoscopic_view_configs = load_yaml(os.path.join(endoscopic_view_model_prefix, 'config.yml'))
non_endoscopic_view_configs = load_yaml(os.path.join(non_endoscopic_view_model_prefix, 'config.yml'))

endoscopic_view_model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(endoscopic_view_model_prefix, endoscopic_view_model), shape=endoscopic_view_configs['model']['shape'])
non_endoscopic_view_model = DeepImageHomographyEstimationModuleBackbone.load_from_checkpoint(os.path.join(non_endoscopic_view_model_prefix, non_endoscopic_view_model), shape=non_endoscopic_view_configs['model']['shape'])


device = 'cpu'
if torch.cuda.is_available():
    print('Running with CUDA backend.')
    device = 'cuda'

endoscopic_view_model.to(device)
endoscopic_view_model = endoscopic_view_model.eval()
endoscopic_view_model.freeze()

non_endoscopic_view_model.to(device)
non_endoscopic_view_model = non_endoscopic_view_model.eval()
non_endoscopic_view_model.freeze()


/home/martin/.local/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


Running with CUDA backend.


In [2]:
import numpy as np
from kornia.geometry import warp_perspective, resize

from utils.viz import yt_alpha_blend
from utils.processing import four_point_homography_to_matrix, image_edges

# load sample video
data_prefix = '/media/martin/Samsung_T5/data/endoscopic_data/cholec80/videos'
video = 'video01.mp4'
vc = cv2.VideoCapture(os.path.join(data_prefix, video))

n = 0
increment = 10

while vc.isOpened():


    # load
    vc.set(cv2.CAP_PROP_POS_FRAMES, n)
    _, img = vc.read()
    n += increment
    vc.set(cv2.CAP_PROP_POS_FRAMES, n)
    _, wrp = vc.read()

    # pre-process
    img = image_to_tensor(np.ascontiguousarray((img[...,::-1])), False).float()/255.
    wrp = image_to_tensor(np.ascontiguousarray((wrp[...,::-1])), False).float()/255.
    img, wrp = resize(img, [240, 320]).to(device), resize(wrp, [240, 320]).to(device)

    # inference
    endoscopic_view_duv = endoscopic_view_model(img, wrp)
    non_endoscopic_view_duv = non_endoscopic_view_model(img, wrp)

    # post-process
    uv = image_edges(img)
    endoscopic_view_H = four_point_homography_to_matrix(uv, -endoscopic_view_duv)
    non_endoscopic_view_H = four_point_homography_to_matrix(uv, -non_endoscopic_view_duv)
    endoscopic_view_wrp_pred = warp_perspective(img, endoscopic_view_H, img.shape[-2:])
    non_endoscopic_view_wrp_pred = warp_perspective(img, non_endoscopic_view_H, img.shape[-2:])
    wrp, endoscopic_view_wrp_pred, non_endoscopic_view_wrp_pred = resize(wrp, [480, 640]), resize(endoscopic_view_wrp_pred, [480, 640]), resize(non_endoscopic_view_wrp_pred, [480, 640])

    # visualize
    endoscopic_view_blend = yt_alpha_blend(wrp, endoscopic_view_wrp_pred)
    non_endoscopic_view_blend = yt_alpha_blend(wrp, non_endoscopic_view_wrp_pred)
    endoscopic_view_blend = tensor_to_image(endoscopic_view_blend, False)
    non_endoscopic_view_blend = tensor_to_image(non_endoscopic_view_blend, False)
    cv2.imshow('endoscopic_view_blend', endoscopic_view_blend)
    cv2.imshow('non_endoscopic_view_blend', non_endoscopic_view_blend)
    cv2.waitKey()

cv2.destroyAllWindows()

/home/martin/miniconda3/envs/torch19/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/martin/miniconda3/envs/torch19/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/martin/miniconda3/envs/torch19/lib/python3.8/site-packages/kornia/utils/helpers.py:96: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyT